In [1508]:
import requests
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

In [1509]:
def download_esios_id(id,fecha_ini,fecha_fin,agrupacion):
                       token = '496b263791ef0dcaf80b803b47b332a13b01f2c2352e018b624c7a36a0eaffc0'
                       cab = dict()
                       cab ['x-api-key']= token
                       url_id = 'https://api.esios.ree.es/indicators'
                       url=f'{url_id}/{id}?geo_ids[]=3&start_date={fecha_ini}T00:00:00&end_date={fecha_fin}T23:59:59&time_trunc={agrupacion}'
                       print(url)
                       response = requests.get(url, headers=cab).json()
                       

                       return response
                       

In [1510]:
id='600'
fecha_ini='2024-01-01'
fecha_fin='2024-12-31'
agrupacion='hour'

In [1511]:
datos_origen =download_esios_id(id,fecha_ini,fecha_fin,agrupacion)


https://api.esios.ree.es/indicators/600?geo_ids[]=3&start_date=2024-01-01T00:00:00&end_date=2024-12-31T23:59:59&time_trunc=hour


In [1512]:
datos=pd.DataFrame(datos_origen['indicator']['values'])
datos

,value,datetime,datetime_utc,tz_time,geo_id,geo_name
0,63.33,2024-01-01T00:00:00.000+01:00,2023-12-31T23:00:00Z,2023-12-31T23:00:00.000Z,3,España
1,50.09,2024-01-01T01:00:00.000+01:00,2024-01-01T00:00:00Z,2024-01-01T00:00:00.000Z,3,España
2,47.50,2024-01-01T02:00:00.000+01:00,2024-01-01T01:00:00Z,2024-01-01T01:00:00.000Z,3,España
3,43.50,2024-01-01T03:00:00.000+01:00,2024-01-01T02:00:00Z,2024-01-01T02:00:00.000Z,3,España
4,42.50,2024-01-01T04:00:00.000+01:00,2024-01-01T03:00:00Z,2024-01-01T03:00:00.000Z,3,España
...,...,...,...,...,...,...
4482,63.31,2024-07-05T19:00:00.000+02:00,2024-07-05T17:00:00Z,2024-07-05T17:00:00.000Z,3,España
4483,82.70,2024-07-05T20:00:00.000+02:00,2024-07-05T18:00:00Z,2024-07-05T18:00:00.000Z,3,España
4484,95.00,2024-07-05T21:00:00.000+02:00,2024-07-05T19:00:00Z,2024-07-05T19:00:00.000Z,3,España
4485,106.33,2024-07-05T22:00:00.000+02:00,2024-07-05T20:00:00Z,2024-07-05T20:00:00.000Z,3,España


In [1513]:
datos = (datos
         .assign(datetime=lambda vh_: pd #formateamos campo fecha, desde un str con diferencia horaria a un naive
                      .to_datetime(vh_['datetime'],utc=True)  # con la fecha local
                      .dt
                      .tz_convert('Europe/Madrid')
                      .dt
                      .tz_localize(None)
                ) 
             #.drop(['datetime','datetime_utc','tz_time','geo_id','geo_name'],
             #      axis=1) #eliminamos campos
             
             .loc[:,['datetime','value']]
             )

datos

,datetime,value
0,2024-01-01 00:00:00,63.33
1,2024-01-01 01:00:00,50.09
2,2024-01-01 02:00:00,47.50
3,2024-01-01 03:00:00,43.50
4,2024-01-01 04:00:00,42.50
...,...,...
4482,2024-07-05 19:00:00,63.31
4483,2024-07-05 20:00:00,82.70
4484,2024-07-05 21:00:00,95.00
4485,2024-07-05 22:00:00,106.33


In [1514]:
meses = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

In [1515]:
datos['fecha']=datos['datetime'].dt.strftime('%d/%m/%Y')
datos['fecha']=pd.to_datetime(datos['fecha'],format='%d/%m/%Y')
datos['hora']=datos['datetime'].dt.hour
datos['dia']=datos['datetime'].dt.day
datos['mes']=datos['datetime'].dt.month
datos['año']=datos['datetime'].dt.year
#datos['mes_nombre']=datos['datetime'].dt.month_name()
#datos['mes_nombre'] = datos['mes_nombre'].map(meses)

datos.set_index('datetime', inplace=True)
datos

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024
...,...,...,...,...,...,...
2024-07-05 19:00:00,63.31,2024-07-05,19,5,7,2024
2024-07-05 20:00:00,82.70,2024-07-05,20,5,7,2024
2024-07-05 21:00:00,95.00,2024-07-05,21,5,7,2024


In [1516]:
datos_dia=datos.resample('D').mean()
datos_dia['value']=datos_dia['value'].round(2)
datos_dia

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01,36.24,2024-01-01,11.5,1.0,1.0,2024.0
2024-01-02,40.91,2024-01-02,11.5,2.0,1.0,2024.0
2024-01-03,55.08,2024-01-03,11.5,3.0,1.0,2024.0
2024-01-04,91.36,2024-01-04,11.5,4.0,1.0,2024.0
2024-01-05,68.62,2024-01-05,11.5,5.0,1.0,2024.0
...,...,...,...,...,...,...
2024-07-01,53.22,2024-07-01,11.5,1.0,7.0,2024.0
2024-07-02,46.97,2024-07-02,11.5,2.0,7.0,2024.0
2024-07-03,61.71,2024-07-03,11.5,3.0,7.0,2024.0


In [1517]:
valor_minimo_diario=datos_dia['value'].min()
valor_maximo_diario=datos_dia['value'].max()
valor_minimo_diario,valor_maximo_diario

(0.44, 113.83)

In [1518]:
def graf_año():
    graf_año=px.line(datos_dia, x='fecha', y='value')
    return graf_año

In [1519]:
graf_año()

In [1520]:
datos_limites = {
    'rango': [-10,20,40,60,80,100,120],
    'valor_asignado': ['muy bajo', 'bajo','medio','alto','muy alto','chungo','xtrem'],
}

In [1521]:
df_limites=pd.DataFrame(datos_limites)
df_limites

,rango,valor_asignado
0,-10,muy bajo
1,20,bajo
2,40,medio
3,60,alto
4,80,muy alto
5,100,chungo
6,120,xtrem


In [1522]:
etiquetas = df_limites['valor_asignado'][:-1]

In [1523]:
datos_dia['escala']=pd.cut(datos_dia['value'],bins=df_limites['rango'],labels=etiquetas,right=False)
datos_dia

,value,fecha,hora,dia,mes,año,escala
datetime,,,,,,,
2024-01-01,36.24,2024-01-01,11.5,1.0,1.0,2024.0,bajo
2024-01-02,40.91,2024-01-02,11.5,2.0,1.0,2024.0,medio
2024-01-03,55.08,2024-01-03,11.5,3.0,1.0,2024.0,medio
2024-01-04,91.36,2024-01-04,11.5,4.0,1.0,2024.0,muy alto
2024-01-05,68.62,2024-01-05,11.5,5.0,1.0,2024.0,alto
...,...,...,...,...,...,...,...
2024-07-01,53.22,2024-07-01,11.5,1.0,7.0,2024.0,medio
2024-07-02,46.97,2024-07-02,11.5,2.0,7.0,2024.0,medio
2024-07-03,61.71,2024-07-03,11.5,3.0,7.0,2024.0,alto


In [1524]:
colores = {
    'muy bajo': 'lightgreen',
    'bajo': 'green',
    'medio': 'blue',
    'alto': 'orange',
    'muy alto': 'red',
    'chungo': 'purple'
}

In [1525]:
datos_dia['color']=datos_dia['escala'].map(colores)
datos_dia

,value,fecha,hora,dia,mes,año,escala,color
datetime,,,,,,,,
2024-01-01,36.24,2024-01-01,11.5,1.0,1.0,2024.0,bajo,green
2024-01-02,40.91,2024-01-02,11.5,2.0,1.0,2024.0,medio,blue
2024-01-03,55.08,2024-01-03,11.5,3.0,1.0,2024.0,medio,blue
2024-01-04,91.36,2024-01-04,11.5,4.0,1.0,2024.0,muy alto,red
2024-01-05,68.62,2024-01-05,11.5,5.0,1.0,2024.0,alto,orange
...,...,...,...,...,...,...,...,...
2024-07-01,53.22,2024-07-01,11.5,1.0,7.0,2024.0,medio,blue
2024-07-02,46.97,2024-07-02,11.5,2.0,7.0,2024.0,medio,blue
2024-07-03,61.71,2024-07-03,11.5,3.0,7.0,2024.0,alto,orange


### Esta tabla se usa para el gráfico grande de barras anual

In [1526]:
escala_ordenada= ['chungo', 'muy alto','alto','medio','bajo','muy bajo']
datos_dia['escala']=pd.Categorical(datos_dia['escala'],categories=escala_ordenada, ordered=True)
datos_dia

,value,fecha,hora,dia,mes,año,escala,color
datetime,,,,,,,,
2024-01-01,36.24,2024-01-01,11.5,1.0,1.0,2024.0,bajo,green
2024-01-02,40.91,2024-01-02,11.5,2.0,1.0,2024.0,medio,blue
2024-01-03,55.08,2024-01-03,11.5,3.0,1.0,2024.0,medio,blue
2024-01-04,91.36,2024-01-04,11.5,4.0,1.0,2024.0,muy alto,red
2024-01-05,68.62,2024-01-05,11.5,5.0,1.0,2024.0,alto,orange
...,...,...,...,...,...,...,...,...
2024-07-01,53.22,2024-07-01,11.5,1.0,7.0,2024.0,medio,blue
2024-07-02,46.97,2024-07-02,11.5,2.0,7.0,2024.0,medio,blue
2024-07-03,61.71,2024-07-03,11.5,3.0,7.0,2024.0,alto,orange


### Esta gráfica se usa para el quesito

In [1527]:
datos_dia_queso=datos_dia.groupby(['escala'])['escala'].count()
datos_dia_queso=datos_dia_queso.reset_index(name='num_dias')
datos_dia_queso


,escala,num_dias
0,chungo,3
1,muy alto,19
2,alto,28
3,medio,43
4,bajo,32
5,muy bajo,62


### Gráfico de barras principal

In [1528]:
def graf_ecv_anual():
    graf_ecv_anual=px.bar(datos_dia, x='fecha', y='value', 
        color='escala',
        color_discrete_map=colores,
        category_orders={'escala':escala_ordenada},
        labels={'value':'precio medio diario €/MWh', 'escala':'escala_cv'},
        title="Precios medios del mercado diario OMIE. Año 2024")
    graf_ecv_anual.update_xaxes(
        showgrid=True
    )
    graf_ecv_anual.update_traces(
        marker_line_width=0
    )

    return graf_ecv_anual

In [1529]:
graf_ecv_anual()

### Gráfico de queso

In [1530]:
def graf_ecv_anual_queso():
    graf_ecv_anual_queso=px.pie(datos_dia_queso, values='num_dias', names='escala',
        color='escala',
        color_discrete_map=colores,
        #marker=dict(colors=colores),
        category_orders={'escala':escala_ordenada},
        labels={'num_dias':'num_dias', 'escala':'escala_cv'},
        title="% y número de días según la Escala CV. Año 2024",
        width=500
        )
    #graf_ecv_anual_queso.update_traces(hoverlabel=dict(font_color='white', bgcolor='blue'))
    #graf_ecv_anual.update_xaxes(
        #tickmode='array',
        #tickvals=pd.date_range(start='2024-01-01', periods=20, freq='MS'),  # Marcas mensuales
        #ticktext=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'],
    #    showgrid=True
    #)
    #graf_ecv_anual.update_traces(
    #    marker_line_width=0
    #)

    return graf_ecv_anual_queso

In [1531]:
graf_ecv_anual_queso()

### Gráfica horaria anual

In [1532]:
datos

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024
...,...,...,...,...,...,...
2024-07-05 19:00:00,63.31,2024-07-05,19,5,7,2024
2024-07-05 20:00:00,82.70,2024-07-05,20,5,7,2024
2024-07-05 21:00:00,95.00,2024-07-05,21,5,7,2024


In [1533]:
pt_curva_horaria=datos.pivot_table(
    values='value',
    index='hora'
)
pt_curva_horaria=pt_curva_horaria['value'].round(2)
pt_curva_horaria=pt_curva_horaria.reset_index()


In [1534]:
pt_curva_horaria

,hora,value
0,0,46.15
1,1,41.50
2,2,39.21
3,3,37.97
4,4,37.13
5,5,38.90
6,6,44.21
7,7,54.21
8,8,51.32
9,9,39.57


In [1554]:
graf_horaria=px.line(pt_curva_horaria,x='hora',y='value',
                     title="Precios medios horarios",
                     labels={'value': '€/MWh'},
                     width=800)
graf_horaria

In [1536]:
mes=1
filtro_mes=datos['mes']==mes
datos_filtrados=datos[filtro_mes]
filtro_mes

datetime
2024-01-01 00:00:00     True
2024-01-01 01:00:00     True
2024-01-01 02:00:00     True
2024-01-01 03:00:00     True
2024-01-01 04:00:00     True
                       ...  
2024-07-05 19:00:00    False
2024-07-05 20:00:00    False
2024-07-05 21:00:00    False
2024-07-05 22:00:00    False
2024-07-05 23:00:00    False
Name: mes, Length: 4487, dtype: bool

In [1537]:
datos_filtrados

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01 00:00:00,63.33,2024-01-01,0,1,1,2024
2024-01-01 01:00:00,50.09,2024-01-01,1,1,1,2024
2024-01-01 02:00:00,47.50,2024-01-01,2,1,1,2024
2024-01-01 03:00:00,43.50,2024-01-01,3,1,1,2024
2024-01-01 04:00:00,42.50,2024-01-01,4,1,1,2024
...,...,...,...,...,...,...
2024-01-31 19:00:00,119.47,2024-01-31,19,31,1,2024
2024-01-31 20:00:00,168.35,2024-01-31,20,31,1,2024
2024-01-31 21:00:00,94.54,2024-01-31,21,31,1,2024


In [1538]:
#datos_filtrados.set_index('datetime', inplace=True)
#datos_filtrados

In [1539]:
datos_filtrados.dtypes

value           float64
fecha    datetime64[ns]
hora              int32
dia               int32
mes               int32
año               int32
dtype: object

In [1540]:
datos_filtrados_dia=datos_filtrados.resample('D').mean()
datos_filtrados_dia

,value,fecha,hora,dia,mes,año
datetime,,,,,,
2024-01-01,36.242500,2024-01-01,11.5,1.0,1.0,2024.0
2024-01-02,40.907917,2024-01-02,11.5,2.0,1.0,2024.0
2024-01-03,55.084167,2024-01-03,11.5,3.0,1.0,2024.0
2024-01-04,91.357917,2024-01-04,11.5,4.0,1.0,2024.0
2024-01-05,68.615417,2024-01-05,11.5,5.0,1.0,2024.0
2024-01-06,43.970000,2024-01-06,11.5,6.0,1.0,2024.0
2024-01-07,75.998333,2024-01-07,11.5,7.0,1.0,2024.0
2024-01-08,105.509167,2024-01-08,11.5,8.0,1.0,2024.0
2024-01-09,113.828750,2024-01-09,11.5,9.0,1.0,2024.0


In [1541]:
def graf_dia():
    graf_dia=px.line(datos_filtrados_dia,x='fecha',y='value')
    
    return graf_dia

In [1542]:
graf_dia()